In [1]:
import numpy as np
import pandas as pd
from itertools import chain, combinations
import json
from tqdm import tqdm
import math

In [2]:
# gets the graphs created inside "creatingGraphs"
df = pd.read_csv('linkedGraphs.csv', engine='python', header=None) 

In [3]:
# deleting unrelevant stuff and renaming
df = df.drop([0], axis = 0)
df = df.rename(columns={0: "Matrix"})

In [4]:
df

,Matrix
1,"[[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]..."
2,"[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]..."
3,"[[1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]..."
4,"[[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0]..."
5,"[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]..."
...,...
19996,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]..."
19997,"[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]..."
19998,"[[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]..."
19999,"[[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]..."


In [5]:
size_of_matrixs = len(json.loads(df.iloc[0]["Matrix"]))
amount_of_matrixs = len(df)

In [6]:
pset_dict = dict()
'''
Returns the powerset of the list given
Since this function can take time, we keep a cache, "pset_dict"
that will keep the powersets we already calculated.
I use the len of the graph as a key, because this powerset
function is only used in sets that starts with 0 and increase their
elements by 1.
'''
def powerset(s):
    x = len(s)
    if x in pset_dict:
        return pset_dict[x]
    pset = []
    for i in range(1 << x):
        pset.append([s[j] for j in range(x) if (i & (1 << j))])
    pset_dict[x] = pset
    return pset

In [7]:
'''
Gets the minimum vertex cover of a given graph.
'''
def get_min_vertex_cover(graph):
    vert = [i for i in range(size_of_matrixs)]
    subsets = list(powerset(vert))
    subsets = sorted(subsets, key=len) 
    current_subset = []
    for subset in subsets:
        # choose the version needed
        if is_vertex_cover2(graph, subset):
            return subset  

In [8]:
'''
Checks if the subset given is a vertex cover for the graph given.
'''
def is_vertex_cover(graph, subset):
    graph_visited = len(graph) * [False]
    # for each vertex in subset, mark as true
    for i in subset: 
        graph_visited[i] = True
    # for each edge, checks that one of the vertex(at least) is True.
    for u in range(int(len(graph) / size_of_matrixs)):
        for v in range(u):
            if graph[u*size_of_matrixs + v] == 1 and not graph_visited[u] and not graph_visited[v]:
                return False
    return True

In [9]:
'''
Checks if the subset given is a vertex cover for the graph given.

This version is used when we use a new df that we pulled from "creatingGraphs"
if we want to create new solutions to a df where the graphs are already represented
as one array, we use the other version
'''
def is_vertex_cover2(graph, subset):
    graph_visited = len(graph) * [False]
    # for each vertex in subset, mark as true
    for i in subset: 
        graph_visited[i] = True
    # for each edge, checks that one of the vertex(at least) is True.
    for u in range(len(graph)):
        for v in range(u):
            if graph[u][v] == 1 and not graph_visited[u] and not graph_visited[v]:
                return False
    return True

In [10]:
sols = []
# solves the problem
for ix in tqdm(df.index):
    current = df.iloc[ix-1]
    sol = get_min_vertex_cover(json.loads(current['Matrix']))
    sols.append(sol)

100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [15:26<00:00, 21.58it/s]


In [11]:
# adds the solution to the df
df['Solution'] = sols

In [12]:
df

,Matrix,Solution
1,"[[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]...","[0, 2, 3, 4, 6, 9, 12, 13]"
2,"[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[1, 2, 3, 5, 7, 8, 14]"
3,"[[1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]...","[0, 3, 5, 6, 9, 11, 12]"
4,"[[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0]...","[0, 1, 4, 5, 7, 8, 13]"
5,"[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]...","[0, 1, 2, 4, 8, 10, 13]"
...,...,...
19996,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...","[2, 4, 6, 7, 9, 11]"
19997,"[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[1, 2, 5, 6, 8, 10, 12]"
19998,"[[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]...","[0, 1, 6, 7, 8, 10, 11, 14]"
19999,"[[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]...","[2, 4, 6, 8, 10, 11, 12, 13]"


In [13]:
'''
Gets an array with numbers(vertex) and returns a new array as
hot one encoding in the size of the matrix.
'''
def hot_one_encoding(array):
    new_array = [0 for i in range(size_of_matrixs)]
    for i in array:
        new_array[i] = 1
    return new_array

In [14]:
new_sols = [0 for i in range(amount_of_matrixs)]
# turns the solutions to hot one encoding 
for i in tqdm(range(len(new_sols))):
    new_sols[i] = hot_one_encoding(sols[i])
new_sols

100%|████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 626651.53it/s]


[[1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
 [0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0],
 [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0],
 [1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0],
 [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1],
 [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1],
 [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0],
 [1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
 [0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0],
 [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [15]:
df['Solution'] = new_sols

In [16]:
df

,Matrix,Solution
1,"[[1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1]...","[1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0]"
2,"[[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...","[0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1]"
3,"[[1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]...","[1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]"
4,"[[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0]...","[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0]"
5,"[[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]...","[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0]"
...,...,...
19996,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...","[0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]"
19997,"[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...","[0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]"
19998,"[[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1]...","[1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1]"
19999,"[[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]...","[0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0]"


In [17]:
new_matrix = [0 for i in range(amount_of_matrixs)]
# transform the matrix to 1d list instead of 2d list
for i in tqdm(range(amount_of_matrixs)):
    current = df.iloc[i]
    flat_list = [item for sublist in json.loads(current['Matrix']) for item in sublist]
    new_matrix[i] = flat_list

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:02<00:00, 9232.22it/s]


In [18]:
df["Matrix"] =  new_matrix

In [19]:
amount_of_sol = [0 for i in range(amount_of_matrixs)]
# counts the amount of vertex in the solution found.
for i in tqdm(range(amount_of_matrixs)):
    current = df.iloc[i]
    amount = sum(current['Solution'])
    amount_of_sol[i] = amount

100%|█████████████████████████████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 12363.44it/s]


In [20]:
df["Amount"] =  amount_of_sol

In [21]:
# save the new df in "linkedGraphsWithSolutions"
df.to_csv('linkedGraphsWithSolutions.csv', index=False)